## Notebook to process GNSS data for one receiver

### Step 1: Load libraries

Chunk that sloads necessary packages & sets working environment to where the jupyterlab notebook file is 

In [1]:
%load_ext autoreload
%autoreload 2
import gnssvod as gv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pdb
import importlib
import zipfile
import os
import xarray as xr
import glob
import shutil
from gnssvod.hemistats.hemistats import hemibuild
#import georinex as gr
#import qgrid as interactive table 
from matplotlib.collections import PatchCollection
import matplotlib.dates as mdates
import shutil
from matplotlib import rcParams
from datetime import datetime, timedelta

### Step 2: Time intervals

Set up time intervals for the study period. The argument **periods** represents the number of days from the start day.

In [6]:
# Define the start day
startday = pd.to_datetime('21-03-2025', format='%d-%m-%Y')
# Generate a range of datetime values
timeintervals=pd.interval_range(start=startday, periods=10, freq='D', closed='left')
timeintervals

IntervalIndex([[2025-03-21 00:00:00, 2025-03-22 00:00:00),
               [2025-03-22 00:00:00, 2025-03-23 00:00:00),
               [2025-03-23 00:00:00, 2025-03-24 00:00:00),
               [2025-03-24 00:00:00, 2025-03-25 00:00:00),
               [2025-03-25 00:00:00, 2025-03-26 00:00:00),
               [2025-03-26 00:00:00, 2025-03-27 00:00:00),
               [2025-03-27 00:00:00, 2025-03-28 00:00:00),
               [2025-03-28 00:00:00, 2025-03-29 00:00:00),
               [2025-03-29 00:00:00, 2025-03-30 00:00:00),
               [2025-03-30 00:00:00, 2025-03-31 00:00:00)],
              dtype='interval[datetime64[ns], left]')

### Step 3: Read RINEX file

We then proceed to read RINEX file. First we will run the snippet of code to perform the pre-processing in python and visualize the dataframe. The **interval** property is resampling the file to reduce it size, from 1 observation per second to one every 15 s.

Sometimes we could get the following error:

ValueError: Missing an approximate antenna position. Provide the argument ‘approx_position’ to preprocess()

In [11]:
pattern = {'YoungPine-2':'/Users/ger/Library/CloudStorage/Box-Box/Project_MetoliusGNSS/VOD/Data/GNSS/extracted/youngpine/youngpine_pheno/Reach_raw_20250311193931.25O'}
#approx_position=[-4705.036,43.000,23011766.990]
#gv.preprocess(pattern,interval='15s',keepvars=keepvars, approx_position=approx_position)# if you want to use the approximate position after providing coordinates, uncomment this line
result = gv.preprocess(pattern,interval='15s',outputresult=True) # preprocess the data and save the result
obs = result['YoungPine-2'][0] # create observation object 

Created a temporary directory at /var/folders/51/nh4w5ktd20v0jyx9rlm3cnq80000gn/T/tmpx44qucr8
/Users/ger/Library/CloudStorage/Box-Box/Project_MetoliusGNSS/VOD/Data/GNSS/extracted/youngpine/youngpine_pheno/Reach_raw_20250311193931.25O exists | Reading...
Processed 3086161 out of 3161604 lines (97.6%)
Observation file  /Users/ger/Library/CloudStorage/Box-Box/Project_MetoliusGNSS/VOD/Data/GNSS/extracted/youngpine/youngpine_pheno/Reach_raw_20250311193931.25O  is read in 91.15 seconds.
Processing 3075204 individual observations
Calculating Azimuth and Elevation
This file does not exist: /var/folders/51/nh4w5ktd20v0jyx9rlm3cnq80000gn/T/tmpx44qucr8/GFZ0MGXRAP_20250700000_01D_05M_ORB.SP3
Downloading: ftp://gssc.esa.int/gnss/products/2357//GFZ0MGXRAP_20250700000_01D_05M_ORB.SP3.gz

GFZ0MGXRAP_20250700000_01D_05M_ORB.SP3.gz: 1.05MB [00:02, 394kB/s]                             


 | Download completed for ftp://gssc.esa.int/gnss/products/2357//GFZ0MGXRAP_20250700000_01D_05M_ORB.SP3.gz
/var/folders/51/nh4w5ktd20v0jyx9rlm3cnq80000gn/T/tmpx44qucr8/GFZ0MGXRAP_20250700000_01D_05M_ORB.SP3 file is read in 3.08 seconds
This file does not exist: /var/folders/51/nh4w5ktd20v0jyx9rlm3cnq80000gn/T/tmpx44qucr8/GFZ0MGXRAP_20250710000_01D_05M_ORB.SP3
Downloading: ftp://gssc.esa.int/gnss/products/2357//GFZ0MGXRAP_20250710000_01D_05M_ORB.SP3.gz

GFZ0MGXRAP_20250710000_01D_05M_ORB.SP3.gz: 1.04MB [00:04, 253kB/s]                             


 | Download completed for ftp://gssc.esa.int/gnss/products/2357//GFZ0MGXRAP_20250710000_01D_05M_ORB.SP3.gz
/var/folders/51/nh4w5ktd20v0jyx9rlm3cnq80000gn/T/tmpx44qucr8/GFZ0MGXRAP_20250710000_01D_05M_ORB.SP3 file is read in 4.62 seconds
This file does not exist: /var/folders/51/nh4w5ktd20v0jyx9rlm3cnq80000gn/T/tmpx44qucr8/GFZ0MGXRAP_20250700000_01D_30S_CLK.CLK
Downloading: GFZ0MGXRAP_20250700000_01D_30S_CLK.CLK.gz

GFZ0MGXRAP_20250700000_01D_30S_CLK.CLK.gz: 4.89MB [00:04, 1.15MB/s]                            


 | Download completed for GFZ0MGXRAP_20250700000_01D_30S_CLK.CLK.gz
/var/folders/51/nh4w5ktd20v0jyx9rlm3cnq80000gn/T/tmpx44qucr8/GFZ0MGXRAP_20250700000_01D_30S_CLK.CLK file is read in 1.12 seconds
This file does not exist: /var/folders/51/nh4w5ktd20v0jyx9rlm3cnq80000gn/T/tmpx44qucr8/GFZ0MGXRAP_20250710000_01D_30S_CLK.CLK
Downloading: GFZ0MGXRAP_20250710000_01D_30S_CLK.CLK.gz

GFZ0MGXRAP_20250710000_01D_30S_CLK.CLK.gz: 4.82MB [00:04, 1.21MB/s]                            


 | Download completed for GFZ0MGXRAP_20250710000_01D_30S_CLK.CLK.gz
/var/folders/51/nh4w5ktd20v0jyx9rlm3cnq80000gn/T/tmpx44qucr8/GFZ0MGXRAP_20250710000_01D_30S_CLK.CLK file is read in 1.27 seconds
SP3 interpolation is done in 1.52 seconds
Removed the temporary directory at /var/folders/51/nh4w5ktd20v0jyx9rlm3cnq80000gn/T/tmpx44qucr8


**Observation objects** contain the following properties

- obs.filename = the name of the source file
- obs.epoch = a datetime indicate the day at the start of the record
- obs.observation = a pandas data frame containing all measurements
- obs.approx_position = the approximate receiver position as provided in the RINEX file [X,Y,Z]
- obs.receiver_type = the receiver type if provided in the RINEX file
- obs.antenna_type = the antenna type if provided in the RINEX file
- obs.interval = the measurement frequency in seconds
- obs.receiver_clock = the receiver clock if provided in the RINEX file
- obs.version = the version of the RINEX file
- obs.observation_types = the observation types reported as columns in obs.observation

We can look at the day when the record started

In [19]:
obs = result['YoungPine-2'][0]
obs.epoch

datetime.date(2025, 3, 12)

Let's now look at the data:

The pandas data frame has a MultIndex that contains both Epoch and SV as indices. The Epoch is the local time of the measurement and the SV is a satellite identification number (also called PRN).

The columns correspond to:
- C# = Pseudorange from the receiver to the satellite, in meters
- L# = Carrier phase, in cycles
- D# = Doppler, in Hz
- S# = Carrier to noise density C/N_0, in dB (receiver-dependent)
And the numbers (S1, S2, etc. ) indicate the corresponding **GNSS frequency**

The azimuth and elevation of the satellite with respect to the receiver are expressed in degrees. Computation speed for the azimuth and elevation can vary according to your hardware. Most of the time is spent interpolating the orbit parameters to the time stamps of each measurement. This is why it is sometimes useful to resample high frequency data (here one measurement per second) to for instance one measurement each 15 seconds.

In [20]:
obs.observation

C1C  C1X           C2C           C2I  C2X  \
Epoch               SV                                                        
2025-03-11 19:39:45 C12           NaN  NaN           NaN  2.142763e+07  NaN   
                    C24           NaN  NaN           NaN  1.977889e+07  NaN   
                    C25           NaN  NaN           NaN  2.340315e+07  NaN   
                    C26           NaN  NaN           NaN  2.085711e+07  NaN   
                    C34           NaN  NaN           NaN  2.346296e+07  NaN   
...                               ...  ...           ...           ...  ...   
2025-03-12 19:39:45 R11  1.861132e+07  NaN  1.861132e+07           NaN  NaN   
                    R12  2.125953e+07  NaN  2.125954e+07           NaN  NaN   
                    R20  2.066789e+07  NaN  2.066789e+07           NaN  NaN   
                    R21  1.863980e+07  NaN  1.863980e+07           NaN  NaN   
                    R22  2.140545e+07  NaN  2.140547e+07           NaN  NaN   

                                  C7I  C7X          D1C  D1X          D2C  \
Epoch               SV                                                      
2025-03-11 19:39:45 C12  2.142764e+07  NaN          NaN  NaN          NaN   
                    C24           NaN  NaN          NaN  NaN          NaN   
                    C25           NaN  NaN          NaN  NaN          NaN   
                    C26           NaN  NaN          NaN  NaN          NaN   
                    C34           NaN  NaN          NaN  NaN          NaN   
...                               ...  ...          ...  ...          ...   
2025-03-12 19:39:45 R11           NaN  NaN    -0.940667  NaN    -0.785333   
                    R12           NaN  NaN  3392.896333  NaN  2638.954333   
                    R20           NaN  NaN -3488.487667  NaN -2713.370167   
                    R21           NaN  NaN   386.551500  NaN   300.629000   
                    R22           NaN  NaN  3955.661667  NaN  3076.850667   

                         ...  L7X        S1C  S1X        S2C     S2I  S2X  \
Epoch               SV   ...                                                
2025-03-11 19:39:45 C12  ...  NaN        NaN  NaN        NaN  40.000  NaN   
                    C24  ...  NaN        NaN  NaN        NaN  47.000  NaN   
                    C25  ...  NaN        NaN  NaN        NaN  30.125  NaN   
                    C26  ...  NaN        NaN  NaN        NaN  38.000  NaN   
                    C34  ...  NaN        NaN  NaN        NaN  40.000  NaN   
...                      ...  ...        ...  ...        ...     ...  ...   
2025-03-12 19:39:45 R11  ...  NaN  48.000000  NaN  41.333333     NaN  NaN   
                    R12  ...  NaN  48.000000  NaN  38.333333     NaN  NaN   
                    R20  ...  NaN  28.166667  NaN  33.000000     NaN  NaN   
                    R21  ...  NaN  45.833333  NaN  39.000000     NaN  NaN   
                    R22  ...  NaN  37.000000  NaN  35.000000     NaN  NaN   

                            S7I  S7X     Azimuth  Elevation  
Epoch               SV                                       
2025-03-11 19:39:45 C12  32.875  NaN -151.998184  42.136821  
                    C24     NaN  NaN  -65.031228  70.864884  
                    C25     NaN  NaN  -50.064981  16.205980  
                    C26     NaN  NaN  141.477563  52.924464  
                    C34     NaN  NaN -141.643568  15.457579  
...                         ...  ...         ...        ...  
2025-03-12 19:39:45 R11     NaN  NaN   51.536772  78.399800  
                    R12     NaN  NaN  -41.740340  28.093224  
                    R20     NaN  NaN   34.975038  35.921406  
                    R21     NaN  NaN  135.911136  79.903612  
                    R22     NaN  NaN -159.957439  26.204451  

[206659 rows x 30 columns]

### Step 4: Saving processed RINEX file

Repeat **Step 3** but instead of creating an object we save the netcdf file in the box folder. This recquires to indicate the location of the file (**pattern**) and the location of the output directory (**outputdir**). 

In [21]:
pattern = {'YoungPine-2':'/Users/ger/Library/CloudStorage/Box-Box/Project_MetoliusGNSS/VOD/Data/GNSS/extracted/youngpine/youngpine_pheno/Reach_raw_20250311193931.25O'}
outputdir={'YoungPine-2':'/Users/ger/Library/CloudStorage/Box-Box/Project_MetoliusGNSS/VOD/Data/GNSS/extracted/youngpine/youngpine_pheno/youngpine_pheno_nc/'}
#approx_position=[-4705.036,43.000,23011766.990]
#gv.preprocess(pattern,interval='15s',keepvars=keepvars,outputdir=outputdir, approx_position=approx_position)# if you want to use the approximate position after providing coordinates, uncomment this line
gv.preprocess(pattern,interval='15s',outputdir=outputdir,outputresult=True) # preprocess the data and save the result as a netcdf file

Created a temporary directory at /var/folders/51/nh4w5ktd20v0jyx9rlm3cnq80000gn/T/tmp602_ju7g
Reach_raw_20250311193931.nc already exists, skipping.. (pass overwrite=True to overwrite)
Removed the temporary directory at /var/folders/51/nh4w5ktd20v0jyx9rlm3cnq80000gn/T/tmp602_ju7g


{'YoungPine-2': []}